In [1]:
!pip install --upgrade huggingface_hub langchain langchain_community

In [2]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceHub
from huggingface_hub import InferenceClient
import os
from huggingface_hub import InferenceClient
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline


In [3]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
CHROMA_DIR = os.path.join(BASE_DIR, "chroma_db")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)
print(BASE_DIR)

C:\Users\bhara\AppData\Local\Temp\ipykernel_27896\1744295691.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\bhara\AppData\Local\Temp\ipykernel_27896\1744295691.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=CHROMA_DIR, embedding_function=embeddings)


C:\Users\bhara\Documents\help_bot


In [4]:
llm = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)

Device set to use cpu


In [5]:
def retriever_agent(query, k=2):
    results = db.similarity_search(query, k=k)
    return [doc.page_content for doc in results]

In [6]:
def summarizer_agent(text_list):
    combined_text = "\n".join(text_list)
    prompt = f"Summarize the following text:\n{combined_text}"
    summary = llm(prompt, max_new_tokens=300)
    return summary[0]['generated_text']

In [19]:
def answer_agent(question, summary):
    prompt = f"Based on the summary below, answer the question in 3 lines atleast. Make sure to give the answer step wise.\n\nSummary:\n{summary}\n\nQuestion: {question}"
    answer = llm(prompt, max_new_tokens=200)
    return answer[0]['generated_text'] 

In [17]:
# ---- 5. Run the Multi-Agent Pipeline ----
query = "i am getting error code ui-800-3. what should i do?"
retrieved_docs = retriever_agent(query)
summary = summarizer_agent(retrieved_docs)
answer = answer_agent(query, summary)


In [18]:
print("Retrieved Docs:", retrieved_docs)
print("\nSummary:", summary)
print("\nAnswer:", answer)

Retrieved Docs: ['If the error code on your device shows only ui-800-3, it means information stored on your device needs to be refreshed, or a network issue stopped Netflix from opening.\n\nTo fix the problem, follow the steps for your device.\n\nTV or TV-connected device\n\nThese steps can fix this error on your TV or a device that connects to your TV, like a streaming stick, media player, set-top box, or Blu-ray player.\n\nRestart your device\n\nRestart your home network\n\nSign out of Netflix', '1. Check your internet connection\n\n\n2. Restart your device\n\n\n3. Restart your home network\n\n\n4. Move your Wi-Fi router\n\n\n5. Set DNS to automatic (PlayStation/Xbox only)\n\nNetflix Error ui-800-3\nIf the error code shown on your device also has numbers in parentheses ( ), search for that exact error code in our Help Center to get the right steps. Example: ui-800-3 (123456)']

Summary: Follow the steps for your device. Restart your device Restart your home network Sign out of Netfli